In [83]:
import pandas as pd
import numpy as np
import pandas as pd
from datetime import datetime
import yfinance as yf

import warnings

try:

    df_list = []

    for i in range(1, 31):
        try:
            filename = f'resources/transactions_summary ({i}).csv'
            df = pd.read_csv(filename)
            df_list.append(df)
            print(f'Successfully loaded file {filename}')
            # Perform any data analysis or manipulation you need
        except FileNotFoundError:
            pass

    shake_data_df = pd.concat(df_list, ignore_index=True)
    shake_data_df['Date'] = pd.to_datetime(shake_data_df['Date']).dt.strftime('%Y-%m-%d')

except FileNotFoundError:
    print("File not found. Please check the filename and path and try again.")

Successfully loaded file resources/transactions_summary (2).csv
Successfully loaded file resources/transactions_summary (3).csv


In [84]:
shake_data_df

,Transaction Type,Date,Amount Debited,Debit Currency,Amount Credited,Credit Currency,Buy / Sell Rate,Direction,Spot Rate,Source / Destination,Blockchain Transaction ID
0,fiat funding,2022-02-22,NaN,NaN,1.000000e+02,CAD,NaN,credit,NaN,yansunique@yahoo.ca,NaN
1,purchase/sale,2022-02-22,100.000000,CAD,2.045910e-03,BTC,48877.8439,purchase,NaN,NaN,NaN
2,crypto cashout,2022-02-22,0.002046,BTC,NaN,NaN,NaN,debit,48477.6729,34FtHDuAGapGYJF7mPAj5Ed4NHFZEcXrQM,722344fd7b6e9ba2a952616cfa0992cb48a2e0608696b6...
3,shakingsats,2022-02-22,NaN,NaN,1.000000e-06,BTC,NaN,credit,48461.8017,NaN,NaN
4,other,2022-02-22,NaN,NaN,3.000000e+01,CAD,NaN,credit,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
7180,purchase/sale,2023-10-13,6.000000,CAD,1.614200e-04,BTC,37168.3152,purchase,NaN,NaN,NaN
7181,shakesquads,2023-10-13,NaN,NaN,8.400000e-07,BTC,NaN,credit,36573.9131,NaN,NaN
7182,purchase/sale,2023-10-14,50.000000,CAD,1.342510e-03,BTC,37243.5712,purchase,NaN,NaN,NaN
7183,crypto cashout,2023-10-14,0.002221,BTC,NaN,NaN,NaN,debit,36780.3821,3LM5RbECKDbVcWVW2sGChMR9oxKhcMavQs,4a4b15c6d50689508e3262c7c2dbf1376a6470b1e4092c...


In [93]:
# assuming you have a datetime column named "date" in the shake_data_df dataframe
start_date = '2021-05-01'
end_date = '2023-09-30'

date_mask = (shake_data_df['Date'] >= start_date) & (shake_data_df['Date'] <= end_date)
filtered_data = shake_data_df.loc[date_mask]

# Create a boolean mask for the desired values
mask2 = filtered_data['Transaction Type'].isin(['shakingstats', 'card cashbacks', 'shakesquads'])

# Apply the mask to the DataFrame to get the filtered rows
filtered_df2 = filtered_data[mask2]
# Group the filtered dataframe by 'Amount credited' and find the sum, and by 'Spot rate' to find the mean
grouped_df = filtered_df2.groupby('Transaction Type').agg({'Amount Credited': 'sum', 'Spot Rate': 'mean'})

# Print the resulting grouped dataframe
# Create a new column 'Value' that multiplies 'Amount credited' and 'Spot rate'
grouped_df['Value'] = grouped_df['Amount Credited'] * grouped_df['Spot Rate']

# Group the dataframe by 'Transaction Type' where 'Transaction Type' is equal to 'shakingsats'
shakingstats = filtered_data[filtered_data['Transaction Type'] == 'shakingsats']


# Group the dataframe by 'Transaction Type' where 'Transaction Type' is equal to 'shakingsats'
shakingstats = filtered_data[filtered_data['Transaction Type'] == 'shakingsats'].groupby('Transaction Type').agg({'Amount Credited': 'sum', 'Spot Rate': 'mean'})

# Print the new DataFrame
shakingstats['Value'] = shakingstats['Amount Credited'] * shakingstats['Spot Rate']
merged = pd.concat([grouped_df,shakingstats])

shake_data_df1 = filtered_data.drop(['Debit Currency','Direction', 'Source / Destination', 'Blockchain Transaction ID'], axis=1)
new_shake_data_df = shake_data_df1[(shake_data_df1['Transaction Type'] == 'purchase/sale') & (shake_data_df1['Credit Currency'] == 'BTC')]
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    new_shake_data_df['weightedbuy'] = new_shake_data_df['Buy / Sell Rate'] * new_shake_data_df['Amount Credited']


weighted_average = new_shake_data_df['weightedbuy'].sum() / new_shake_data_df['Amount Credited'].sum()

## Group By BTC Total

# Filter the DataFrame based on Transaction Type and Credit Currency
BTC_Total = filtered_data[(filtered_data['Transaction Type'] == 'purchase/sale') & (filtered_data['Credit Currency'] == 'BTC')]
BTC_Total = BTC_Total[BTC_Total['Transaction Type'] == 'purchase/sale'].groupby('Transaction Type').agg({'Amount Debited': 'sum', 'Amount Credited': 'sum', 'Buy / Sell Rate': 'mean'})


## Current Price
import yfinance as yf
btc_df = yf.Ticker('BTC-CAD').history(period='7y',interval='1d',actions=False).reset_index()
btc_df = btc_df.iloc[-1].Close

## Stack

# get input from user and convert to float
# my_float_value = float(input("Please enter your float value: "))
BTC_Amount = BTC_Total.iloc[0,1]
# # multiply DataFrame by user input and store in Stack variable
Current_Value_weightedavg = BTC_Amount * weighted_average
DCA = BTC_Total.iloc[0,2]
Total_Invested = BTC_Total.iloc[0,0]
## Current Price

btc_df = yf.Ticker('BTC-CAD').history(period='7y',interval='1d',actions=False).reset_index()
Current_Price = btc_df.iloc[-1].Close

## Return
RETURN_DCA = (Current_Price/DCA)-1
RETURN_WA = (Current_Price/weighted_average)-1

print(f"From {start_date} to {end_date}.")
print(f"Current Price: {(Current_Price):.2f}")
print(f"Total Invested: {Total_Invested:.2f}")
print(f"Current Return DCA: {(RETURN_DCA * 100):.2f}%")
print(f"Average DCA Price: {DCA:.2f}")
print(f"Current Return Weighted Average: {(RETURN_WA * 100):.2f}%")
print(f"Weighted Average Price: {weighted_average:.2f}")
print(f"The total value from card cashbacks is {merged.iloc[0,2]:.2f}")
print(f"The total value from card shakesquads is {merged.iloc[1,2]:.2f}")
print(f"The total value from card shakingsats is {merged.iloc[2,2]:.2f}")

From 2021-05-01 to 2023-09-30.
Current Price: 36764.14
Total Invested: 26674.19
Current Return DCA: 1.20%
Average DCA Price: 36327.11
Current Return Weighted Average: -21.83%
Weighted Average Price: 47028.44
The total value from card cashbacks is 728.40
The total value from card shakesquads is 22.05
The total value from card shakingsats is 132.92
